In [ ]:
from HelmholtzSolverRAD import *
from ExampleBoundaries import sphere_rad

frequency = 40.0 # frequency [Hz]
k = frequency_to_wavenumber(frequency)

interior_points = np.array([[0.0000,  0.0000],
                           [0.0000,  0.5000],
                           [0.0000, -0.5000],
                           [0.5000,  0.0000]], dtype=np.float32)


# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#
solver = InteriorHelmholtzSolverRAD(sphere_rad())

boundary_condition = solver.dirichlet_boundary_condition()
boundary_condition.f[:] = np.sin(k * solver.centers[:,1])

boundary_incidence = BoundaryIncidence(solver.len())
boundary_incidence.phi.fill(0.0)
boundary_incidence.v.fill(0.0)

interior_incident_phi = np.zeros(interior_points.shape[0], dtype=np.complex64)

boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("Test Problem 1")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 2
# von Neumann boundary condition such that phi = sin(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal
boundary_condition = solver.neumann_boundary_condition()
centers = solver.geometry.centers()
normals = solver.geometry.normals()
for i in range(centers.shape[0]):
    z = centers[i, 1]
    n = -normals[i]
    boundary_condition.f[i] = k * np.cos(k*z) * n[1]
    
boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("\n\nTest Problem 2")
print("==============\n")
print(boundary_solution)
print(sample_solution)


# Test Problem 3
# Dirichlet boundary condition, such that phi = sin(k/ sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal
boundary_condition.alpha.fill(1.0)
boundary_condition.beta.fill(0.0)

zp = 0.25
for i in range(centers.shape[0]):
    r = centers[i, 0]
    z = centers[i, 1]
    # make input complex so proper sqrt is called
    rpq = np.sqrt(0.0j + r**2 + (z - zp)**2)
    boundary_condition.f[i]   = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
    boundary_incidence.phi[i] = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
    n = -normals[i]
    drbdn = (r * n[0] + (z - zp) * n[1]) / rpq
    boundary_incidence.v[i] = drbdn * np.exp(1j * k * rpq) * (1j * k * rpq - 1.0) \
        / (4.0 * np.pi * rpq*rpq)
    
for i in range(interior_points.shape[0]):
    r = interior_points[i, 0]
    z = interior_points[i, 1]
    # make input complex so proper sqrt is called
    rpq = np.sqrt(0.0j + r**2 + (zp - z)**2)
    interior_incident_phi[i] = np.exp(1j * k * rpq) / (4.0 * np.pi * rpq)
       
boundary_solution = solver.solve_boundary(k, boundary_condition, boundary_incidence)
sample_solution = boundary_solution.solve_samples(interior_incident_phi, interior_points)

print("\n\nTest Problem 3")
print("==============\n")
print(boundary_solution)
print(sample_solution)

Copyright (C) 2017&ndash;2019 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.